## Matriz inversa
Para que uma matriz $ A $ possua uma matriz inversa $ A^{-1} $, é necessário que $ A $ seja quadrada (mesmo número de linhas e colunas).

A matriz inversa $ A^{-1} $ multiplicada pela matriz $ A $ deve resultar em uma matriz identidade $ I $.

$$ A.A^{-1} = A^{-1}.A = I $$


Podemos então encontrar a matriz inversa $ A^{-1} $ com a seguinte fórmula:

$$ A^{-1} = \frac{1}{det(A)} adj(A) $$

onde:
* $ det(A) $: é a determinante da matriz $ A $;
* $ adj(A) $: é a matriz adjunta da matriz $ A $.

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def check_tamanho_matriz(matriz):
    """
    Função para exibir retorno de erro caso a matriz de entrada:
        1. Não for quadrada
        2. ter dimensão diferente de 2x2 ou 3x3
    """
    lin, col = matriz.shape
    
    if (lin!=col): 
        return {
            'success': False
            , 'message': 'Matriz deve ser quadrada'
        }
    
    if (lin < 2 or lin > 3):
        return {
            'success': False
            , 'message': 'Apenas matriz de tamanho 2x2 ou 3x3 são aceitas'
        }
    
    return{
        'success': True
        , 'message': 'Tamanho da matriz: {}x{}'.format(lin, col)
    }

In [3]:
def calc_determinante(matriz):
    """
    Função que calcula o determinante de uma matriz de dimensão 2x2 ou 3x3.
    """
    lin, col = matriz.shape
    tamanho = check_tamanho_matriz(matriz)
    
    if not tamanho.get('success'):
        return {
            'success': False
            , 'message': tamanho.get('message')
            , 'result': None
        }
    
    if (lin == 2):
        return {
            'success': True
            , 'message': 'Determinante obtida com sucesso.'
            , 'result': ( matriz[0][0] * matriz[1][1] ) - ( matriz[0][1] * matriz[1][0] )
        }
    
    if (lin == 3):
        det = 0
        for i in range(lin):
            det = det + (matriz[0][i] * (matriz[1][(i+1)%3] * matriz[2][(i+2)%3] - matriz[1][(i+2)%3] * matriz[2][(i+1)%3]));
        return {
            'success': True
            , 'message': 'Determinante obtida com sucesso.'
            , 'result': det
        }

In [4]:
def calc_adjunta(matriz):
    """
    Função que calcula a matriz adjunta uma matriz de dimensão 2x2 ou 3x3.
    """
    lin, col = matriz.shape
    tamanho = check_tamanho_matriz(matriz)
    
    if not tamanho.get('success'):
        return {
            'success': False
            , 'message': tamanho.get('message')
            , 'result': None
        }
    
    if (lin == 2):
        adj = np.array([
                [matriz[1][1], -matriz[1][0]],
                [-matriz[0][1], matriz[0][0]]
            ])
    
    if (lin == 3):
        adj = np.zeros((3,3))
        
        for i in range(lin):
            for j in range(col):
                adj[i][j] = calc_determinante(np.array([
                    [matriz[(i+1)%3][(j+1)%3], matriz[(i+1)%3][(j+2)%3]],
                    [matriz[(i+2)%3][(j+1)%3], matriz[(i+2)%3][(j+2)%3]]
                ])).get('result', None) 
    
    return {
            'success': True
            , 'message': 'Adjunta obtida com sucesso.'
            , 'result': np.transpose(adj)
        }

In [5]:
def calc_inversa(matriz):
    """
    Função que calcula a matriz inversa de uma matriz de dimensão 2x2 ou 3x3.
    """
    lin, col = matriz.shape
    tamanho = check_tamanho_matriz(matriz)
    
    if not tamanho.get('success'):
        return {
            'success': False
            , 'message': tamanho.get('message')
            , 'result': None
        }
    
    return {
        'success': True
        , 'message': 'Inversa obtida com sucesso'
        , 'result': (1/calc_determinante(matriz).get('result')) * calc_adjunta(matriz).get('result')
    }

In [6]:
"""
Testando as funções acima.
Input inicial de uma matriz 3x3.
"""

matriz = np.array([
    [5, 2, -2],
    [3, 1, 7],
    [2, 1, 3],  
])

print('Matriz de entrada: \n {} \n\n'.format(matriz))

det = calc_determinante(matriz)
det_res = (det.get('result', None))
print(det.get('message', None))
print((det_res or '-'))
print('\n')

adj = calc_adjunta(matriz)
adj_res = adj.get('result', None)
print(adj.get('message', None))
print(adj_res) if adj_res is not None else print('-')
print('\n')
           
inv = calc_inversa(matriz)
inv_res = inv.get('result', None)
print(inv.get('message', None))
print(inv_res) if inv_res is not None else print('-')
print('\n')

print('Multiplicação A*A^-1')
print(np.dot(matriz,inv_res)) if inv_res is not None else print('-')
    

Matriz de entrada: 
 [[ 5  2 -2]
 [ 3  1  7]
 [ 2  1  3]] 


Determinante obtida com sucesso.
-12


Adjunta obtida com sucesso.
[[ -4.  -8.  16.]
 [  5.  19. -41.]
 [  1.  -1.  -1.]]


Inversa obtida com sucesso
[[ 0.33333333  0.66666667 -1.33333333]
 [-0.41666667 -1.58333333  3.41666667]
 [-0.08333333  0.08333333  0.08333333]]


Multiplicação A*A^-1
[[  1.00000000e+00  -1.38777878e-16   3.05311332e-16]
 [  1.11022302e-16   1.00000000e+00  -2.22044605e-16]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


## Mínimos quadrados

$$ \beta = (X^T X)^{-1} X^Ty $$

In [7]:
def calc_min_quadrado(X, y, quadratico=False):
    lin, col = X.shape
    
    if (quadratico):
        X = np.array([np.append(i, i[col-1]*i[col-1]) for i in X])
    
    inv = calc_inversa(np.dot(np.transpose(X), X))
    
    if not inv.get('success'):
        return {
            'success': False
            , 'message': 'Não foi possível calcular o mínimo quadrado.\n {}'.format(inv.get('message'))
            , 'result': None
        }
    
    return {
        'success': True
        , 'message': 'Mínimo quadrado {} calculado com sucesso.'.format('(quadratico)' if quadratico else '')
        , 'result': np.dot(inv.get('result'),np.dot(np.transpose(X), y))
    }

### Height x Shoe size

In [8]:
X = np.array([
    [1, 69],
    [1, 67],
    [1, 71],
    [1, 65],
    [1, 72],
    [1, 68],
    [1, 74],
    [1, 65],
    [1, 66],
    [1, 72]
])

y = np.array([
    [9.5],
    [8.5],
    [11.5],
    [10.5],
    [11],
    [7.5],
    [12],
    [7],
    [7.5],
    [13]
])

min_quad = calc_min_quadrado(X, y)
min_quad_res = min_quad.get('result', None)
print(min_quad.get('message', None))
print(min_quad_res) if min_quad_res is not None else print('-')
print('\n')

min_quad = calc_min_quadrado(X, y, True)
min_quad_res = min_quad.get('result', None)
print(min_quad.get('message', None))
print(min_quad_res) if min_quad_res is not None else print('-')
print('\n')



Mínimo quadrado  calculado com sucesso.
[[-25.65123789]
 [  0.51453175]]


Mínimo quadrado (quadratico) calculado com sucesso.
[[ -4.21942130e+06]
 [ -5.40741892e+00]
 [  4.28048409e-02]]




### Boiling point at the Alps

In [9]:
BPt = np.array([
    [1, 194.5],
    [1, 194.3],
    [1, 197.9],
    [1, 198.4],
    [1, 199.4],
    [1, 199.9],
    [1, 200.9],
    [1, 201.1],
    [1, 201.4],
    [1, 201.3],
    [1, 203.6],
    [1, 204.6],
    [1, 209.5],
    [1, 208.6],
    [1, 210.7],
    [1, 211.9],
    [1, 212.2]    
])

Pressure = np.array([
    [20.79],
    [20.79],
    [22.4],
    [22.67],
    [23.15],
    [23.35],
    [23.89],
    [23.99],
    [24.02],
    [24.01],
    [25.14],
    [26.57],
    [28.49],
    [27.76],
    [29.04],
    [29.88],
    [30.06]
])

min_quad = calc_min_quadrado(BPt, Pressure)
min_quad_res = min_quad.get('result', None)
print(min_quad.get('message', None))
print(min_quad_res) if min_quad_res is not None else print('-')
print('\n')

min_quad = calc_min_quadrado(BPt, Pressure, True)
min_quad_res = min_quad.get('result', None)
print(min_quad.get('message', None))
print(min_quad_res) if min_quad_res is not None else print('-')
print('\n')

Mínimo quadrado  calculado com sucesso.
[[-81.06372713]
 [  0.5228924 ]]


Mínimo quadrado (quadratico) calculado com sucesso.
[[  3.88292988e+01]
 [ -6.54770851e-01]
 [  2.88971271e-03]]




### Books x Grades

In [10]:
books_attends = np.array([
    [1, 0, 9],
    [1, 1, 15],
    [1, 0, 10],
    [1, 2, 16],
    [1, 4, 10],
    [1, 4, 20],
    [1, 1, 11],
    [1, 4, 20],
    [1, 3, 15],
    [1, 0, 15],
    [1, 2, 8],
    [1, 1, 13],
    [1, 4, 18],
    [1, 1, 10],
    [1, 0, 8],
    [1, 1, 10],
    [1, 3, 16],
    [1, 0, 11],
    [1, 1, 19],
    [1, 4, 12],
    [1, 4, 11],
    [1, 0, 19],
    [1, 2, 15],
    [1, 3, 15],
    [1, 1, 20],
    [1, 0, 6],
    [1, 3, 15],
    [1, 3, 19],
    [1, 2, 14],
    [1, 2, 13],
    [1, 3, 17],
    [1, 2, 20],
    [1, 2, 11],
    [1, 3, 20],
    [1, 4, 20],
    [1, 4, 20],
    [1, 3, 9],
    [1, 1, 8],
    [1, 2, 16],
    [1, 0, 10]
])

grades = np.array([
    [45],
    [57],
    [45],
    [51],
    [65],
    [88],
    [44],
    [87],
    [89],
    [59],
    [66],
    [65],
    [56],
    [47],
    [66],
    [41],
    [56],
    [37],
    [45],
    [58],
    [47],
    [64],
    [97],
    [55],
    [51],
    [61],
    [69],
    [79],
    [71],
    [62],
    [87],
    [54],
    [43],
    [92],
    [83],
    [94],
    [60],
    [56],
    [88],
    [62]
])

min_quad = calc_min_quadrado(books_attends, grades)
min_quad_res = min_quad.get('result', None)
print(min_quad.get('message', None))
print(min_quad_res) if min_quad_res is not None else print('-')
print('\n')

min_quad = calc_min_quadrado(books_attends, grades, True)
min_quad_res = min_quad.get('result', None)
print(min_quad.get('message', None))
print(min_quad_res) if min_quad_res is not None else print('-')
print('\n')

Mínimo quadrado  calculado com sucesso.
[[ 37.3791852 ]
 [  4.03689261]
 [  1.28347727]]


Não foi possível calcular o mínimo quadrado.
 Apenas matriz de tamanho 2x2 ou 3x3 são aceitas
-




### US Census

In [11]:
X = np.array([
    [1, 1900],
    [1, 1910],
    [1, 1920],
    [1, 1930],
    [1, 1940],
    [1, 1950],
    [1, 1960],
    [1, 1970],
    [1, 1980],
    [1, 1990],
    [1, 2000]
])

y = np.array([
    [75.9950],
    [91.9720],
    [105.7110],
    [123.2030],
    [131.6690],
    [150.6970],
    [179.3230],
    [203.2120],
    [226.5050],
    [249.6330],
    [281.4220]
])

min_quad = calc_min_quadrado(X, y)
min_quad_res = min_quad.get('result', None)
print(min_quad.get('message', None))
print(min_quad_res) if min_quad_res is not None else print('-')
print('\n')

min_quad = calc_min_quadrado(X, y, True)
min_quad_res = min_quad.get('result', None)
print(min_quad.get('message', None))
print(min_quad_res) if min_quad_res is not None else print('-')
print('\n')



Mínimo quadrado  calculado com sucesso.
[[ -3.78394559e+03]
 [  2.02530273e+00]]


Mínimo quadrado (quadratico) calculado com sucesso.
[[ -2.44524585e+09]
 [  2.75670665e+09]
 [  6.95532552e+05]]


